# Now Building CKD model after spliting the data into training and test part 
### (We can split the data into training, test and validation part but the observation points is too small)


### Load all the relavent libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from imblearn.over_sampling import SMOTE

In [6]:
#! pip install imblearn

### Load the Consolidated CKD Dataset for further processing

In [2]:
CKD = pd.read_csv("consolidate_ckd.csv")

In [3]:
CKD.head()

,id,race,gender,age,creatinine_no_of_test,creatinine_mean_value,creatinine_duration_of_test,glucose_no_of_test,glucose_mean_value,glucose_duration_of_test,...,pitavastatin,pravastatin,propranolol,rosuvastatin,simvastatin,telmisartan,valsartan,total_no_of_days_with_drug,total_no_of_days_without_drug,Stage_Progress
0,0,Unknown,Male,70,6,1.236667,2043,6,6.456667,2043,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1290.0,328.0,True
1,1,White,Female,71,5,1.838000,1214,5,9.240000,1602,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300.0,221.0,False
2,2,White,Female,57,5,2.170000,1737,3,6.466667,804,...,NaN,NaN,NaN,180.0,NaN,NaN,NaN,180.0,487.0,True
3,3,White,Male,74,5,0.996000,1800,6,6.525000,2475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.0,301.0,False
4,4,White,Female,50,6,1.150000,2120,7,9.720000,1974,...,NaN,720.0,NaN,NaN,NaN,NaN,NaN,1620.0,481.0,True


In [5]:
CKD.Stage_Progress.value_counts()

False    200
True     100
Name: Stage_Progress, dtype: int64

In [1]:
CKD.fillna(0)

NameError: name 'CKD' is not defined